<a href="https://colab.research.google.com/github/aryanlalwani/BIOL7409Final/blob/main/ichanWT2005_FIcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install NEURON

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
repo_name = 'BIOL7409Final'

if 'google.colab' in str(get_ipython()): # Running on colab
    import os        
    # from distutils.dir_util import copy_tree
    if not os.path.exists(repo_name):
        !git clone https://github.com/aryanlalwani/{repo_name}.git # downloads repository into our Google colab session's file system        
    
    os.chdir('/content/' + repo_name)   
!nrnivmodl mechanisms2

/content/BIOL7409Final
Mod files: "mechanisms2/mechanisms2/cad.mod" "mechanisms2/mechanisms2/ca.mod" "mechanisms2/mechanisms2/gabaa.mod" "mechanisms2/mechanisms2/ichan_T875M.mod" "mechanisms2/mechanisms2/ichanWT2005.mod" "mechanisms2/mechanisms2/ichanWT.mod" "mechanisms2/mechanisms2/kca.mod" "mechanisms2/mechanisms2/km.mod" "mechanisms2/mechanisms2/kv.mod" "mechanisms2/mechanisms2/na12.mod" "mechanisms2/mechanisms2/na16.mod" "mechanisms2/mechanisms2/na.mod" "mechanisms2/mechanisms2/Nav11.mod" "mechanisms2/mechanisms2/Nav11_T875M.mod"

 -> Compiling mod_func.cpp
 -> Compiling ichanWT.c
ichanWT.c: In function ‘rates__ichanWT’:
ichanWT.c:176:15: error: too few arguments to function ‘vtrap_ichanWT’
  176 | #define vtrap vtrap_ichanWT
      |               ^~~~~~~~~~~~~
ichanWT.c:373:23: note: in expansion of macro ‘vtrap’
  373 |    _lalpha = - 0.07 * vtrap ( _threadargscomma_ ( _lv - 47.0 ) / - 6.0 ) ;
      |                       ^~~~~
ichanWT.c:176:15: note: declared here
  176 | #defi

In [22]:
from neuron import h
from neuron import gui
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

Example of current injection in single compartment cell with Mainen delayed rectifier potassium current. Note units are different for kv and nav11. Kv uses pS/µm2 and Nav1.1 uses S/cm2 (1 S/cm^2 = 10,000 pS/µm2).

In [19]:
soma = h.Section(name='soma')
soma.diam = 25
soma.L = 25
soma.nseg = 1

# Biophysics
soma.insert("ichanWT.mod")
soma.enat = 50
soma.ekf = -80
soma.gnatbar_ichanWT2005 = 0.2
soma.gkfbar_ichanWT2005 = 0.16
soma.gl_ichanWT2005 = 0.0005
soma.el_ichanWT2005 = -60

# soma.cm = 1
# soma.insert('pas')
# soma.insert('extracellular')
# soma.g_pas = 5e-4
# soma.e_pas = -60
# soma.insert('nav11')
# soma.gbar_nav11 = 0.2

# soma.insert('kv')
# soma.gbar_kv = 150
# soma.ena = 50
# soma.ek = -80

ic = h.IClamp(soma(0.5))
ic.delay = 50
ic.dur = 200
ic.amp = 0.1

h.v_init = -61 # mV - initial voltage of the model system 
h.dt = 0.005  # ms - time step
h.celsius = 21 # degrees celsius - temperature of simulation, set to room temperature
h.tstop = 250

# sTau_vec = h.Vector().record(soma(0.5)._ref_sTau_nav11)
# hTau_vec = h.Vector().record(soma(0.5)._ref_hTau_nav11)

netcon = h.NetCon(soma(0.5)._ref_v,None, sec=soma) # Use NetCon to monitor axon voltage 
netcon.threshold = 0 # set threshold to detect AP
# Record spike times
spike_times = h.Vector()
netcon.record(spike_times) # pass the spiketimes vector to the record method of the netcon object we just created

amp_vec = np.linspace(0.1, 0.4, 100) # create vector here

F_vec1 = []
for amp in amp_vec:
    ic.amp = amp # nA
    # Run
    h.run()
    Fi = len(spike_times)/(ic.dur*1e-3) # number of spikes divided by length of simulation in sec
    F_vec1.append(Fi)  

# Plotting code goes here
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(amp_vec, F_vec1)
ax.set_ylabel('Frequency')
ax.set_xlabel('Current Amplitude')

ic.amp = 0.1


ValueError: ignored

In [ ]:
h.celsius = 21
ic.amp = 0.4
h.run()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=t,y=V,name='Vm',line=dict(color='rgb(0,0,0)')),secondary_y = False)

fig.add_trace(go.Scatter(x=t,y=m_vec,name='m'),secondary_y = True)
fig.add_trace(go.Scatter(x=t,y=h_vec,name='h'),secondary_y = True)
fig.add_trace(go.Scatter(x=t,y=s_vec,name='s'),secondary_y = True)
fig.add_trace(go.Scatter(x=t,y=sTau_vec/sTau_vec.max(),name='sTau'),secondary_y = True)
# fig.add_trace(go.Scatter(x=t,y=hTau_vec/hTau_vec.max(),name='hTau'),secondary_y = True)
fig.update_layout(yaxis = dict(range=[-80,50]))


Example code to insert the ichanWT2005.mod mechanism that includes their implementation of Nav1.1, with a leak and delayed rectifier (commented, in case you want to use it, replace the code above inserting the nav11, kv, and pas mechanisms with this one)

In [ ]:
soma.insert('ichanWT2005')
soma.enat = 50
soma.ekf = -80
soma.gnatbar_ichanWT2005 = 0.2
soma.gkfbar_ichanWT2005  = 0.06
soma.gl_ichanWT2005 = 0.0005
soma.el_ichanWT2005 = -60